In [1]:
import logging
import os
try:
    from pathlib2 import Path
except ImportError:
    from pathlib import Path

import numpy as np
import torch
import torch.multiprocessing as mp
import parmap

from detect.detector import Detect
from localization_pipeline.denoiser import Denoise

from detect.deduplication import deduplicate_gpu, deduplicate

from scipy.signal import argrelmin

from detect.run import run


In [2]:
geom_path = '/media/cat/julien/spikes_localization_registration/channels_maps/np2_channel_map.npy'
path_nn_detector = '/media/cat/julien/spikes_localization_registration/pretrained_detector/detect_np2.pt'
path_nn_denoiser = '/media/cat/julien/spikes_localization_registration/pretrained_denoiser/denoise.pt'
standardized_path = '/media/cat/julien/nick_drift/different_preprocessings/data_standardized_registered/standardized_unregistered.bin'
standardized_dtype = 'float32'
sampling_rate = 30000
len_recording = 1000
output_directory = 'detection_results_threshold'

geom_array = np.load(geom_path)
apply_nn = False ### If set to false, run voltage threshold instead of NN detector 
spatial_radius = 70
n_sec_chunk = 1
n_processors = 4
n_sec_chunk_gpu_detect = .1
detect_threshold = 6 ## 0.5 if apply NN, 4/5/6 otherwise 
n_filters_detect = [16, 8, 8] 
spike_size_nn = 121 ### In sample steps
n_filters_denoise = [16, 8, 4]
filter_sizes_denoise = [5, 11, 21]
n_batches = len_recording//n_sec_chunk


In [3]:
sampling_rate = 30000

run(standardized_path, standardized_dtype, output_directory, geom_array, spatial_radius, apply_nn, n_sec_chunk, n_batches, n_processors, n_sec_chunk_gpu_detect, sampling_rate, len_recording,
    detect_threshold, path_nn_detector, n_filters_detect, spike_size_nn, path_nn_denoiser, n_filters_denoise, filter_sizes_denoise, run_chunk_sec='full')

   batch length to (sec):  1  (longer increase speed a bit)
   length of each seg (sec):  0.5


  0%|          | 0/2000 [00:05<?, ?it/s]
Process ForkPoolWorker-1:
Traceback (most recent call last):
  File "/home/julien/ENTER/envs/yass_run/lib/python3.7/site-packages/parmap/parmap.py", line 117, in _func_star_many
    **func_items_args[3])
  File "/home/julien/ENTER/envs/yass_run/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/julien/ENTER/envs/yass_run/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/julien/ENTER/envs/yass_run/lib/python3.7/multiprocessing/pool.py", line 121, in worker
    result = (True, func(*args, **kwds))
  File "/home/julien/ENTER/envs/yass_run/lib/python3.7/multiprocessing/pool.py", line 44, in mapstar
    return list(map(*args))
  File "/media/cat/julien/spikes_localization_registration/detect/run.py", line 740, in run_voltage_threshold_parallel
    channel_index)
  File "/media/cat/julien/spikes_localization_registration/detect/deduplicati

KeyboardInterrupt: 

Process ForkPoolWorker-2:
Traceback (most recent call last):
  File "/home/julien/ENTER/envs/yass_run/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/julien/ENTER/envs/yass_run/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/julien/ENTER/envs/yass_run/lib/python3.7/multiprocessing/pool.py", line 121, in worker
    result = (True, func(*args, **kwds))
  File "/home/julien/ENTER/envs/yass_run/lib/python3.7/multiprocessing/pool.py", line 44, in mapstar
    return list(map(*args))
  File "/home/julien/ENTER/envs/yass_run/lib/python3.7/site-packages/parmap/parmap.py", line 117, in _func_star_many
    **func_items_args[3])
  File "/media/cat/julien/spikes_localization_registration/detect/run.py", line 740, in run_voltage_threshold_parallel
    channel_index)
  File "/media/cat/julien/spikes_localization_registration/detect/deduplication.py", line 19, in deduplicate_gpu
    m

In [ ]:
#### LOCALIZATION

import os
import numpy as np
from tqdm import tqdm
# from residual import RESIDUAL
from localization_pipeline.localizer import LOCALIZER
from localization_pipeline.merge_results import get_merged_arrays


In [ ]:
### Change paths to your data here
bin_file = standardized_path
residual_file = bin_file
dtype_input = 'float32'

fname_spike_train = 'detection_results_threshold/spike_index.npy'
# Sort spike train if not 
spt_array = np.load(fname_spike_train)
spt_array = spt_array[spt_array[:, 0].argsort()]
np.save(fname_spike_train, spt_array)

geom_path = "channels_maps/np2_channel_map.npy"
n_channels = np.load(geom_path).shape[0]

denoiser_weights = path_nn_denoiser
denoiser_min = 42 ## Goes with the weights

n_batches = 1000
len_recording = 1000
sampling_rate = 30000

fname_templates = None

In [ ]:
localizer_obj = LOCALIZER(bin_file, residual_file, dtype_input, fname_spike_train, fname_templates, geom_path, denoiser_weights, denoiser_min)
# localizer_obj.get_offsets()
# localizer_obj.compute_aligned_templates()
localizer_obj.load_denoiser()
output_directory = 'position_results_files'
if not os.path.exists(output_directory):
    os.makedirs(output_directory)
for i in tqdm(range(n_batches)):
    localizer_obj.get_estimate(i, threshold = 6, output_directory = 'position_results_files')

